In [1]:
from app.db import DatabaseConnection
from app.request import load
from dotenv import load_dotenv 
import pandas as pd
import numpy as np
import ijson
import os

params = {
    "host": os.environ.get("POSTGRES_HOST") ,
    "database": os.environ.get("POSTGRES_DB") ,
    "user": os.environ.get("POSTGRES_DB") ,
    "password": os.environ.get("POSTGRES_PASSWORD") ,
    "port": os.environ.get("POSTGRES_PORT") 
}

conn = DatabaseConnection(**params)
conn.connect()

In [7]:
def create_load_data(
    shema_name: str = "public",
    table_name: str = None, 
    path_file: str = None,
    conn = None
    ) -> None:
    
    if not table_name:
        table_name = os.path.basename(path_file).split(".")[0]
    
    df = pd.read_json(path_file)
    dtype = df.dtypes
    del df
    
    for name, ty in dtype.copy().items():
        if ty == int:
            dtype[name] = "INTEGER"
        elif ty == float:
            dtype[name] = "FLOAT"
        elif ty == bool:
            dtype[name] = "BOOLEAN"
        elif ty == object:
            dtype[name] = "TEXT"
        elif ty == np.datetime64:
            dtype[name] = "DATE"
        else:
            dtype[name] = "UNKNOWN"
    
    content_table = "id SERIAL PRIMARY KEY, "
    for name, value in dtype.items():
        content_table += f"{name} {value}, "
    content_table = content_table[:-2]
    
    query = f"CREATE TABLE IF NOT EXISTS {shema_name}.{table_name}({content_table})"
    conn.execute(query)
    conn.commit()
    
    with open(path_file, "r", encoding="utf-8") as json_file:
        
        objets_json = ijson.items(json_file, "item")
        
        query = f"INCERT INTO {table_name} {', '.join(dtype.keys())}" 
        copy = query
        
        for obj in objets_json:
            
            params = []
            for val in obj.values():
                
                if val == object:
                    params.append(str(val))
                else:
                    params.append(val)
            print(params)
            query += f"{', '.join(['%s'] * len(params))}"
            conn.execute(query, params)
            query = copy
    
    return query

In [8]:
create_load_data("request_4d4b0081_65f9_4681_bac7_dfe3f205e54e","test", "trash/data/New_collector_20231214_154733.success.json", conn)

('464087281009', '2.9', '47', 'Footwear', '70% off', 'bucik', 'https://www.ajio.com/bucik-cross-strappy-sandals/p/464087281_brown', ['j_lq5dbaxrzw1czqfic.c055587471519cfa3f5265a4ddb4202bf3d91720.file_lq5dcice1prj6ai89w.[_].jpg', 'j_lq5dbaxrzw1czqfic.c055587471519cfa3f5265a4ddb4202bf3d91720.file_lq5dcice224prtfomx.[_].jpg', 'j_lq5dbaxrzw1czqfic.c055587471519cfa3f5265a4ddb4202bf3d91720.file_lq5dcice28pe1zgdrr.[_].jpg'], '660', 'INR', '2199', [], 'Men', ['https://assets.ajio.com/medias/sys_master/root/20230621/5fkA/64928264d55b7d0c638f6965/bucik_brown_cross_strappy_sandals.jpg', 'https://assets.ajio.com/medias/sys_master/root/20230621/5fkA/64928264d55b7d0c638f6958/bucik_brown_cross_strappy_sandals.jpg', 'https://assets.ajio.com/medias/sys_master/root/20230621/5fkA/64928264d55b7d0c638f6955/bucik_brown_cross_strappy_sandals.jpg'], {'url': 'https://www.ajio.com/b/bucik', 'category': ''})


ProgrammingError: can't adapt type 'dict'